In [1]:
from qiskit import QuantumCircuit, execute
from qiskit import IBMQ, Aer
from qiskit.visualization import plot_histogram
from qiskit_aer.noise import NoiseModel

from qiskit import *
import numpy as np
from qiskit.visualization import plot_histogram
import qiskit.tools.jupyter
from qiskit.tools.monitor import job_monitor
from qiskit.ignis.mitigation.measurement import *
import matplotlib.pyplot as plt

# below three lines were shifted up from block 2
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram
import qiskit_aer.noise as noise

from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)

In [2]:
noise_model = noise.NoiseModel()

noise_model_TI = noise.NoiseModel() # initialize noise model for Trapped Ion Qubits
noise_model_S = noise.NoiseModel() # initialize noise model for Superconducting Qubits
noise_model_QD = noise.NoiseModel() # initialize noise model for Quantum Dot Qubits
noise_model_CC = noise.NoiseModel() # initialize noise model for Color Center Qubits

In [3]:
def create_noiseModel_TI():

    noise_model_TI = noise.NoiseModel() # initialize noise model for Trapped Ion Qubits
    # Error probabilities: FOR TRAPPED ION QUBITS

    prob_1 = 0.00001/100  # 1-qubit gate, 0.00001% error rate
    prob_2 = 0.1/100   # 2-qubit gate, 0.1% error rate

    # DEPOLARIZING QUANTUM ERRORS
    error_1 = noise.depolarizing_error(prob_1, 1) # set depolarizing error for single-qubit gates
    error_2 = noise.depolarizing_error(prob_2, 2) # set depolarizing error for double-qubit gates

    # Add errors to noise model
    noise_model_TI.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3', 'x'])
    noise_model_TI.add_all_qubit_quantum_error(error_2, ['cx', 'cnot'])

    ## ## ## ## ## ##

    ## THERMAL DECOHERENCE ERROR FOR TRAPPED ION QUBITS

    # T1 and T2 values (decoherence times) for qubits in nanoseconds
    T1s = 600000000 # It said around 10 minutes, so around 6*10^11 nS
    T2s = 2*T1s  # Requirement: T2 <= 2T1

    # Truncate random T2s <= T1s
    # T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(4)])

    # Instruction times (in nanoseconds) # PLACEHOLDER VALUES FOR GATE TIMES
                                            # (within 1000 to 100 000 nS)
    time_u1 = 3000   # virtual gate
    time_u2 = 3000  # (single X90 pulse)
    time_u3 = 3000 # (two X90 pulses)
    time_cx = 100000
    time_reset = 3000  # 1 microsecond
    time_measure = 3000 # 1 microsecond
    time_x = 3000

    # QuantumError objects
    errors_reset = thermal_relaxation_error(T1s, T2s, time_reset)

    errors_measure = thermal_relaxation_error(T1s, T2s, time_measure)

    errors_u1  = thermal_relaxation_error(T1s, T2s, time_u1)

    errors_u2  = thermal_relaxation_error(T1s, T2s, time_u2)

    errors_u3  = thermal_relaxation_error(T1s, T2s, time_u3)

    errors_cx = thermal_relaxation_error(T1s, T2s, time_cx).expand(
                 thermal_relaxation_error(T1s, T2s, time_cx))

    errors_x = thermal_relaxation_error(T1s, T2s, time_x)

    # Add errors to noise model

    noise_model_TI.add_all_qubit_quantum_error(errors_reset, "reset")
    noise_model_TI.add_all_qubit_quantum_error(errors_measure, "measure")
    noise_model_TI.add_all_qubit_quantum_error(errors_u1, "u1")
    noise_model_TI.add_all_qubit_quantum_error(errors_u2, "u2")
    noise_model_TI.add_all_qubit_quantum_error(errors_u3, "u3")

    noise_model_TI.add_all_qubit_quantum_error(errors_x, "x")

    noise_model_TI.add_all_qubit_quantum_error(errors_cx, "cx")

    print(noise_model_TI)
    
    return noise_model_TI

In [4]:
def create_noiseModel_SQ():

    noise_model_S = noise.NoiseModel() # initialize noise model for Superconducting Qubits

    # Error probabilities: FOR SUPERCONDUCTING QUBITS

    prob_1 = 0.2/100  # 1-qubit gate, 0.2% error rate
    prob_2 = 0.5/100   # 2-qubit gate, 0.5% error rate

    # Depolarizing quantum errors
    error_1 = noise.depolarizing_error(prob_1, 1) # set depolarizing error for single-qubit gates
    error_2 = noise.depolarizing_error(prob_2, 2) # set depolarizing error for double-qubit gates

    # Add errors to noise model
    noise_model_S.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3', 'x'])
    noise_model_S.add_all_qubit_quantum_error(error_2, ['cx', 'cnot'])

    ## ## ## ## ##

    ## THERMAL DECOHERENCE ERROR FOR SUPERCONDUCTING QUBITS

    # T1 and T2 values (decoherence times) for qubits in nanoseconds
    T1s = 50000 # It said around 10^-4 seconds (100 000 nS) to 10^-5 seconds (10 000 nS)
    T2s = 2*T1s  # Requirement: T2 <= 2T1

    # Truncate random T2s <= T1s
    # T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(4)])

    # Instruction times (in nanoseconds) # PLACEHOLDER VALUES FOR GATE TIMES
                                            # (within 10 to 100 nS)
    time_u1 = 50  
    time_u2 = 50  
    time_u3 = 50  
    time_cx = 50 
    time_reset = 50   
    time_measure = 50 
    time_x = 50

    # QuantumError objects
    errors_reset = thermal_relaxation_error(T1s, T2s, time_reset)

    errors_measure = thermal_relaxation_error(T1s, T2s, time_measure)

    errors_u1  = thermal_relaxation_error(T1s, T2s, time_u1)

    errors_u2  = thermal_relaxation_error(T1s, T2s, time_u2)

    errors_u3  = thermal_relaxation_error(T1s, T2s, time_u3)

    errors_cx = thermal_relaxation_error(T1s, T2s, time_cx).expand(
                 thermal_relaxation_error(T1s, T2s, time_cx))

    errors_x = thermal_relaxation_error(T1s, T2s, time_x)

    # Add errors to noise model

    noise_model_S.add_all_qubit_quantum_error(errors_reset, "reset")
    noise_model_S.add_all_qubit_quantum_error(errors_measure, "measure")
    noise_model_S.add_all_qubit_quantum_error(errors_u1, "u1")
    noise_model_S.add_all_qubit_quantum_error(errors_u2, "u2")
    noise_model_S.add_all_qubit_quantum_error(errors_u3, "u3")

    noise_model_S.add_all_qubit_quantum_error(errors_x, "x")

    noise_model_S.add_all_qubit_quantum_error(errors_cx, "cx")

    print(noise_model_S)
    
    return noise_model_S

In [5]:
def create_noiseModel_QD():

    noise_model_QD = noise.NoiseModel() # initialize noise model for Quantum Dot Qubits

    # Error probabilities: FOR QUANTUM DOTS

    prob_1 = 0.4/100  # 1-qubit gate, 0.4% error rate
    prob_2 = 2/100   # 2-qubit gate, 2% error rate

    # Depolarizing quantum errors
    error_1 = noise.depolarizing_error(prob_1, 1) # set depolarizing error for single-qubit gates
    error_2 = noise.depolarizing_error(prob_2, 2) # set depolarizing error for double-qubit gates

    # Add errors to noise model
    noise_model_QD.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3', 'x'])
    noise_model_QD.add_all_qubit_quantum_error(error_2, ['cx', 'cnot'])

    ## ## ## ## ##

    ## THERMAL DECOHERENCE ERROR FOR QUANTUM DOT QUBITS

    # T1 and T2 values (decoherence times) for qubits in nanoseconds
    T1s = 150000 # It said around 10^-4 seconds (100 000 nS) to 10^-5 seconds (10 000 nS)
    T2s = 300000  # Requirement: T2 <= 2T1

    # Truncate random T2s <= T1s
    # T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(4)])

    # Instruction times (in nanoseconds) # PLACEHOLDER VALUES FOR GATE TIMES
                                            # (within 10 to 100 nS)
    time_u1 = 100  
    time_u2 = 100  
    time_u3 = 100  
    time_cx = 200 
    time_reset = 100   
    time_measure = 100 
    time_x = 100

    # QuantumError objects
    errors_reset = thermal_relaxation_error(T1s, T2s, time_reset)

    errors_measure = thermal_relaxation_error(T1s, T2s, time_measure)

    errors_u1  = thermal_relaxation_error(T1s, T2s, time_u1)

    errors_u2  = thermal_relaxation_error(T1s, T2s, time_u2)

    errors_u3  = thermal_relaxation_error(T1s, T2s, time_u3)

    errors_cx = thermal_relaxation_error(T1s, T2s, time_cx).expand(
                 thermal_relaxation_error(T1s, T2s, time_cx))

    errors_x = thermal_relaxation_error(T1s, T2s, time_x)

    # Add errors to noise model

    noise_model_QD.add_all_qubit_quantum_error(errors_reset, "reset")
    noise_model_QD.add_all_qubit_quantum_error(errors_measure, "measure")
    noise_model_QD.add_all_qubit_quantum_error(errors_u1, "u1")
    noise_model_QD.add_all_qubit_quantum_error(errors_u2, "u2")
    noise_model_QD.add_all_qubit_quantum_error(errors_u3, "u3")

    noise_model_QD.add_all_qubit_quantum_error(errors_x, "x")

    noise_model_QD.add_all_qubit_quantum_error(errors_cx, "cx")

    print(noise_model_QD)
    
    return noise_model_QD

In [6]:
def create_noiseModel_CC():

    noise_model_CC = noise.NoiseModel() # initialize noise model for Color Center Qubits

    # Error probabilities: FOR COLOR CENTERS

    prob_1 = 0.0048/100  # 1-qubit gate, 0.0048% error rate
    prob_2 = 0.8/100   # 2-qubit gate, 0.8% error rate

    # Depolarizing quantum errors
    error_1 = noise.depolarizing_error(prob_1, 1) # set depolarizing error for single-qubit gates
    error_2 = noise.depolarizing_error(prob_2, 2) # set depolarizing error for double-qubit gates

    # Add errors to noise model
    noise_model_CC.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3', 'x'])
    noise_model_CC.add_all_qubit_quantum_error(error_2, ['cx', 'cnot'])

    # Need to add: 
    # - Noise from:
    #      - State Preparation and Measurement (SPAM) errors
    #      - Dephasing errors? (maybe thermal decoherence will suffice?)
    # - More graphs and stuff to compare?

    ## ## ## ## ##

    ## THERMAL DECOHERENCE ERROR FOR COLOR CENTER QUBITS

    # T1 and T2 values (decoherence times) for qubits in nanoseconds
    T1s = 187500 # It said around 10^-4 seconds (100 000 nS) to 10^-5 seconds (10 000 nS)
    T2s = 375000  # Requirement: T2 <= 2T1

    # Truncate random T2s <= T1s
    # T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(4)])

    # Instruction times (in nanoseconds) # PLACEHOLDER VALUES FOR GATE TIMES
                                            # (within 10 to 100 nS)
    time_u1 = 50  
    time_u2 = 50  
    time_u3 = 50  
    time_cx = 1000 
    time_reset = 50   
    time_measure = 50 
    time_x = 50

    # QuantumError objects
    errors_reset = thermal_relaxation_error(T1s, T2s, time_reset)

    errors_measure = thermal_relaxation_error(T1s, T2s, time_measure)

    errors_u1  = thermal_relaxation_error(T1s, T2s, time_u1)

    errors_u2  = thermal_relaxation_error(T1s, T2s, time_u2)

    errors_u3  = thermal_relaxation_error(T1s, T2s, time_u3)

    errors_cx = thermal_relaxation_error(T1s, T2s, time_cx).expand(
                 thermal_relaxation_error(T1s, T2s, time_cx))

    errors_x = thermal_relaxation_error(T1s, T2s, time_x)

    # Add errors to noise model

    noise_model_CC.add_all_qubit_quantum_error(errors_reset, "reset")
    noise_model_CC.add_all_qubit_quantum_error(errors_measure, "measure")
    noise_model_CC.add_all_qubit_quantum_error(errors_u1, "u1")
    noise_model_CC.add_all_qubit_quantum_error(errors_u2, "u2")
    noise_model_CC.add_all_qubit_quantum_error(errors_u3, "u3")

    noise_model_CC.add_all_qubit_quantum_error(errors_x, "x")

    noise_model_CC.add_all_qubit_quantum_error(errors_cx, "cx")

    print(noise_model_CC)
    
    return noise_model_CC

In [7]:
def basisGatesTI():

    noise_model_TI = create_noiseModel_TI()
    
    basis_gates_TI = noise_model_TI.basis_gates
    
    return basis_gates_TI

In [8]:
def basisGatesSQ():

    noise_model_SQ = create_noiseModel_SQ()
    
    basis_gates_SQ = noise_model_SQ.basis_gates
    
    return basis_gates_SQ

In [9]:
def basisGatesQD():

    noise_model_QD = create_noiseModel_QD()
    
    basis_gates_QD = noise_model_QD.basis_gates
    
    return basis_gates_QD

In [10]:
def basisGatesCC():

    noise_model_CC = create_noiseModel_CC()
    
    basis_gates_CC = noise_model_CC.basis_gates
    
    return basis_gates_CC

In [11]:
# BELOW IS A BLOCK THAT SHOWS HOW TO IMPLEMENT THE FUNCTIONS DEFINED ABOVE INTO YOUR QISKIT CODE:

In [12]:
# Import functions that create noise from this file:

from NoiseModelFunctionListQiskit import *

# Perform a noise simulation:

sim = Aer.get_backend('aer_simulator') # defining the simulation backend

shots = 20000 # set number of shots

# below block gets the result of the circuit being run

# note that create_noiseModel_SQ() can be swapped out for the other create_noiseModel functions defined above

# note that basisGatesSQ() can be swapped out for the other basisGates functions defined above

# "create_noiseModel_SQ()" and "basisGatesSQ()" were just used below for example purposes

# if an altered noise model with more or less types of noise is desired, let me know. Cheers.

noise_model_SQ_counts_qc4 = execute(qc4,                                       # the name of the circuit to run (qc4 used here)
                    sim,                                                       # sim is the backend to run it on (defined above)
                    shots=shots,                                               # shots is the number of runs
                    basis_gates=basisGatesSQ() ,                               # basis_gates from the noise model needed
                    noise_model=create_noiseModel_SQ()).result().get_counts(0) # noise model goes here


plot_histogram(noise_model_SQ_counts_qc4, figsize=(9,5)) # plot histogram of counts

ModuleNotFoundError: No module named 'NoiseModelFunctionListQiskit'